Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

# 1. Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
m = Mystem()
def mod(text):
    lemm_text = "".join(m.lemmatize(text))
    return " ".join(re.sub(r'[^a-zA-Z.:)(;]', ' ', lemm_text).split())

In [5]:
%%time
df['lemm_text'] = df['text'].apply(mod)

CPU times: user 58.5 s, sys: 11.9 s, total: 1min 10s
Wall time: 2min 29s


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
text         159571 non-null object
toxic        159571 non-null int64
lemm_text    159571 non-null object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


Провели лемматизацию и почистили от лишних символов.

# 2. Обучение

### Разбиение на признаки

In [7]:
features = df['lemm_text'].values.astype('U')
target = df['toxic']

In [8]:
features = pd.DataFrame(features, columns=['text'])

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3)

In [12]:
len(df.query('toxic == 0'))/len(df.query('toxic == 1'))

8.834884437596301

На один отрицательныйй отзыв приходится почти 9 положительных

In [13]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [14]:
features_train_upsample, target_train_upsample = upsample(features_train, target_train, 4)

Увеличили кол-во отрицательных отзывово на трейне в 5 раз

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Логистическая регрессия

In [16]:
%%time
vect = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

CPU times: user 34 µs, sys: 8 µs, total: 42 µs
Wall time: 48.4 µs


In [17]:
%%time
vect_train = vect.fit_transform(features_train_upsample['text'].values.astype('U'))

CPU times: user 31.7 s, sys: 2.41 s, total: 34.1 s
Wall time: 34.5 s


In [18]:
%%time
vect_test = vect.transform(features_test['text'].values.astype('U'))

CPU times: user 7.01 s, sys: 581 ms, total: 7.59 s
Wall time: 7.72 s


In [19]:
%%time
model = LogisticRegression(random_state=12345, max_iter=1000)
model.fit(vect_train, target_train_upsample)
predicted = model.predict(vect_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 12min 20s, sys: 3min 16s, total: 15min 36s
Wall time: 15min 46s


In [20]:
f1_score(predicted, target_test)

0.7867400928625418

In [16]:
%%time
tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2), sublinear_tf=True)

CPU times: user 37 µs, sys: 9 µs, total: 46 µs
Wall time: 49.4 µs


In [17]:
%%time
tf_idf_train = tf_idf.fit_transform(features_train_upsample['text'].values.astype('U'))

CPU times: user 33.8 s, sys: 2.1 s, total: 35.9 s
Wall time: 36 s


In [18]:
%%time
tf_idf_test = tf_idf.transform(features_test['text'].values.astype('U'))

CPU times: user 7.51 s, sys: 594 ms, total: 8.1 s
Wall time: 8.13 s


In [19]:
%%time
model = LogisticRegression(random_state=12345, max_iter=1000)
model.fit(tf_idf_train, target_train_upsample)
predicted = model.predict(tf_idf_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 13.9 s, sys: 3.62 s, total: 17.6 s
Wall time: 17.7 s


In [20]:
f1_score(predicted, target_test)

0.7691346454233051

### Случайный лес

In [21]:
model = RandomForestClassifier(max_depth=200, n_estimators=20, random_state=12345)
model.fit(vect_train, target_train_upsample)
predicted = model.predict(vect_test)
f1_score(predicted, target_test)

0.4327028294513052

In [21]:
model = RandomForestClassifier(max_depth=200, n_estimators=20, random_state=12345)
model.fit(tf_idf_train, target_train_upsample)
predicted = model.predict(tf_idf_test)
f1_score(predicted, target_test)

0.3964715316760224

### Модель Xgboost

In [22]:
model = xgb.XGBClassifier(max_depth=10)
model.fit(vect_train, target_train_upsample)
predicted = model.predict(vect_test)
f1_score(predicted, target_test)

0.7207394569612942

In [23]:
model = xgb.XGBClassifier(max_depth=10)
model.fit(tf_idf_train, target_train_upsample)
predicted = model.predict(tf_idf_test)
f1_score(predicted, target_test)

0.7325949367088607

После того как я разбил на тренировочную, валидационную и тестовую выборки и прогнал по некоторым алгоритмам, заметил, что в нашем случае разбиение на валидацию и тест не требуется, т.к. особой вариативности в параметры моделей я не вносил(не видел надобности), а модель случайного леса давала такие плохие метрики качества, что она даже не рассматривалась на тесте. Поэтому я разбил на обучающую и тестовую.

### Увеличение класса

После того как я увеличил количество строк с одним из классов, тренировочной выборки, модели стали выдавать намного лучшую мектрику.

### Преобразование текста

Текст преобразован двумя способами CountVectorizer и TfidfVectorizer, которые очень похожи, только второй более информативный т.к. учитывает частоту появления слов в тексте.

### Логистическая регрессия

Лучше всего показала себя логистическая регрессия. 

### Случайный лес

Алгоритм случайного леса совсем не подошел для нашей задачи, похожую метрику качества можно получить случайным образом, зная только пропорции классов.

### Алгоритм XGBClassifier

Алгоритм показал очень хороший результат, лучшая метрика получилась на основе случайного леса.